# **Problem 1) a)**

In [66]:
# Mounting the Google Drive 
from google.colab import drive
drive.mount('/content/gdrive')

# Setting my input location to the Google Colab
%cd /content/gdrive/My Drive/DIS/Project2/Problem 1

# Displaying contents in the location
!ls

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/DIS/Project2/Problem 1
Box_filter_lena_grayscale.png	     Prewitt_x_filter_wolves.png
Box_filter_wolves.png		     Prewitt_y_filter_lena_grayscale.png
Convoluted_Impulse_Image.png	     Prewitt_y_filter_wolves.png
fo_c_filter_lena_grayscale.png	     Roberts_x_filter_lena_grayscale.png
fo_c_filter_wolves.png		     Roberts_x_filter_wolves.png
fo_r_filter_lena_grayscale.png	     Roberts_y_filter_lena_grayscale.png
fo_r_filter_wolves.png		     Roberts_y_filter_wolves.png
Gray_Impulse_Image.png		     Sobel_x_filter_lena_grayscale.png
lena_grayscale.png		     Sobel_x_filter_wolves.png
lena.png			     Sobel_y_filter_lena_grayscale.png
Padded_lena_grayscale.png	     Sobel_y_filter_wolves.png
Padded_wolves.png		     wolves.png
Prewitt_x_filter_lena_grayscale.png


In [67]:
# Importing the required libraries for the problem
import cv2
from google.colab.patches import cv2_imshow
import numpy as np

In [68]:
# Defining the kernels that will be used for computation

def kernel(i):
# Box filter
  if (i == 1):
    b_mat = np.array([[1, 1, 1],[1,1,1],[1,1,1]],np.int32)
    box_filter = (1/9)*b_mat
    a = 'Box_filter_'
    return box_filter,a

# First order derivative filter
  elif (i == 2):
    fo_r_filter = np.array([-1, 1],np.int32)
    a = 'fo_r_filter_'
    return fo_r_filter,a
  elif (i == 3):
    fo_c_filter = np.array([[1],[-1]],np.int32)
    a = 'fo_c_filter_'
    return fo_c_filter,a

# Prewitt filter
  elif(i == 4):
    pre_x = np.array([[-1, 0, 1],[-1, 0, 1],[-1, 0, 1]],np.int32)
    a = 'Prewitt_x_filter_'
    return pre_x,a
  elif (i == 5):
    pre_y = np.array([[1, 1, 1],[0, 0, 0],[-1, -1, -1]],np.int32)
    a = 'Prewitt_y_filter_'
    return pre_y,a

# Sobel filter
  elif (i == 6):
    sob_x = np.array([[-1, 0, 1],[-2, 0, 2],[-1, 0, 1]],np.int32)
    a = 'Sobel_x_filter_'
    return sob_x,a
  elif (i == 7):
    sob_y = np.array([[1, 2, 1],[0, 0, 0],[-1, -2, -1]],np.int32)
    a = 'Sobel_y_filter_'
    return sob_y,a

# Roberts filter
  elif (i == 8):
    rob_x = np.array([[0, 1],[-1, 0]],np.int32)
    a = 'Roberts_x_filter_'
    return rob_x,a
  elif (i == 9):
    rob_y = np.array([[1, 0],[0, -1]],np.int32)
    a = 'Roberts_y_filter_'
    return rob_y,a

In [69]:
def padding(img,pad,x):  # img - original Image ; pad - type of pad ; x - length of pad on each edge of the image
  r,c,ch = img.shape
  pad_r = r+(2*x)
  pad_c = c+(2*x)
  padded_image = np.zeros((pad_r,pad_c,ch), dtype = np.int32)
  padded_image[x:(pad_r-x),x:(pad_c-x),:] = img
  if (pad == 1):
    # Zero Padding
    return padded_image
  elif (pad==2):
    # Wrap Around Padding
    padded_image[0:x,:,:] = padded_image[pad_r-(2*x)-1:pad_r-x-1,:,:]
    padded_image[pad_r-x:pad_r,:,:] = padded_image[x:2*x,:,:]
    padded_image[:,0:x,:] = padded_image[:,pad_c-(2*x)-1:pad_c-x-1,:]
    padded_image[:,pad_c-x:pad_c,:] = padded_image[:,x:2*x,:]
    return padded_image
  elif (pad==3):
    # Reflection Padding
    padded_image[0:x,:,:] = padded_image[2*x:x:-1,:,:]
    padded_image[pad_r-1:pad_r-x-1:-1,:,:] = padded_image[pad_r-(2*x)-1:pad_r-x-1,:,:]
    padded_image[:,0:x,:] = padded_image[:,2*x:x:-1,:]
    padded_image[:,pad_c-1:pad_c-x-1:-1,:] = padded_image[:,pad_c-(2*x)-1:pad_c-x-1,:]
    return padded_image
  elif (pad==4):
    # Replicate Padding
    padded_image[0:x,:,:] = padded_image[x:x+1,:,:]
    padded_image[pad_r-x:pad_r,:,:] = padded_image[pad_r-x-1:pad_r-x,:,:]
    padded_image[:,0:x,:] = padded_image[:,x:x+1,:]
    padded_image[:,pad_c-x-1:pad_c,:] = padded_image[:,pad_c-x-1:pad_c-x,:]
    return padded_image

In [70]:
def conv2(f,w,x):     # f - padded image, w - kernel
  if w.ndim == 1:
    kernel = np.flip(w)   # flipping a 1D kernel
    r = 1
    c = kernel.shape[0]
  else:
    kernel = np.fliplr(np.flipud(w))    # flipping the 2D kernel for the convolution operation
    r,c = kernel.shape
  height,width = f.shape
  op_image = np.zeros((f.shape[0],f.shape[1]),np.int32)  # defining an ndarray for the output convoluted image
  
  for i in range(height-r-1):
    for j in range(width-c-1):
      mat = f[i:i+r, j:j+c]
      op_image[i][j] = np.sum(np.multiply(mat,kernel))   # convolution function for the padded image
  original_op = op_image[x:(height-x),x:(width-x)]   # extracting the original image from the padded convoluted image
  return original_op

In [71]:
x = int(input("Enter the length of the padding "))    # inputting the length of padding 
print("Enter your choice of padding:")    # choice for padding
pad = int(input(" 1. Zero Padding\n 2. Wrap Around Padding\n 3. Reflect across Padding \n 4. Replicate Padding\n"))
image = ['lena_grayscale.png','wolves.png']     # array for choosing the images
for i in range(len(image)): 
  Image = np.array(cv2.imread(image[i]), dtype = 'int32')   # reading the image
  padded_image = padding(Image,pad,x)       # padding the image using user-defined padding function
  filename = 'Padded_'+image[i]
  cv2.imwrite(filename,padded_image)
  for j in range(1,10):     # loop for looping over all the types of filter
    K,name = kernel(j)      # calling the kernel function
    B,G,R = cv2.split(padded_image)   # splitting the image into its components and calling the convolution function 
    channel_B = conv2(B,K,x)          # for all the three channels
    channel_G = conv2(G,K,x)
    channel_R = conv2(R,K,x)
    output_image = cv2.merge([channel_B,channel_G,channel_R])  # and merging them after convolution
    f_name = name+image[i]
    cv2.imwrite(f_name,output_image)

Enter the length of the padding 10
Enter your choice of padding:
 1. Zero Padding
 2. Wrap Around Padding
 3. Reflect across Padding 
 4. Replicate Padding
1


# **Problem 1)b)**

In [72]:
gray_image = np.zeros((1024,1024),np.int32)     # defining a zeros matrix with size 1024x1024
gray_image[511,511] = 255         # making the middle value to be 255 (max)
cv2.imwrite('Gray_Impulse_Image.png',gray_image)   
subregion_gray = gray_image[505:518,505:518]  # printing few pixel values for comparison
print ('Subregion of original gray image: ')
print(subregion_gray) 

K,name  = kernel(1)
print(K)
 
conv = conv2(gray_image,K,0)        # applying the convolution function from before
cv2.imwrite('Convoluted_Impulse_Image.png',conv)
subregion_conv = conv[505:518,505:518]
print ('Subregion of convoluted gray image: ')
print(subregion_conv)

Subregion of original gray image: 
[[  0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0 255   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0]]
[[0.11111111 0.11111111 0.11111111]
 [0.11111111 0.11111111 0.11111111]
 [0.11111111 0.11111111 0.11111111]]
Subregion of convoluted gray image: 
[[ 0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0